## Downstream prediction using MLP

Perform downstream predictions using different embeddings with an MLP.  Replicate some of the XGB experiments using an MLP downstream model.

In [ ]:
import os
from phase.prediction import *
from sklearn.preprocessing import StandardScaler

PATH  = os.path.expanduser("~/phase/")
DPATH = PATH+"downstream_prediction/"
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"

lookback = 60
DEBUG = False

label_type_eta_currfeat_lst = [("desat_bool92_5_nodesat",0.02,"SAO2"),
                               ("nibpm60",0.1,"NIBPM"), 
                               ("etco235",0.1,"ETCO2")]

for label_type, _, curr_feat in label_type_eta_currfeat_lst:
    print("\n[Progress] label_type: {}, eta: {}, curr_feat {}".format(label_type, "NA", curr_feat))

    xgb_type = "mlp_{}_top15".format(label_type)
    RESDIR = '{}{}/'.format(RESULTPATH, xgb_type)
    if not os.path.exists(RESDIR): os.makedirs(RESDIR)

    for hosp_data in [1]:
        print("\n[Progress] hosp_data {}".format(hosp_data))

        dt_lst = ["ema[top15]+nonsignal",
                  "raw[top15]+nonsignal",
                  "randemb[top15]+nonsignal"]
        for hosp_model in [0,1,"P"]:
            if hosp_model == "P" and "desat" not in label_type: continue
                
            print("\n[Progress] hosp_model {}".format(hosp_model))
            if hosp_model == 0:
                dt_lst += ["nextfive_{}[top15]+nonsignal".format(hosp_model),
                           "auto_{}[top15]+nonsignal".format(hosp_model),
                           "min5_{}[top15]+nonsignal".format(hosp_model)]
            else:
                dt_lst += ["nextfive_{}[top15]+nonsignal".format(hosp_model),
                           "auto_{}[top15]+nonsignal".format(hosp_model),
                           "min5_{}[top15]+nonsignal".format(hosp_model)]

            if label_type.startswith("desat"):
                dt_lst.append("hypox_{}[top15]+nonsignal".format(hosp_model))
            elif label_type.startswith("etco2"):
                dt_lst.append("hypoc_{}[top15]+nonsignal".format(hosp_model))
            elif label_type.startswith("nibpm"):
                dt_lst.append("hypot_{}[top15]+nonsignal".format(hosp_model))

        for data_type in dt_lst:
            print("\n[Progress] data_type {}".format(data_type))
            (trainvalX,trainvalY) = load_data(DPATH,data_type,label_type,True,
                                              hosp_data,curr_feat,DEBUG=DEBUG)
            print("[Progress] trainvalX.shape {}".format(trainvalX.shape))
            # Standardize data
            if "raw" in data_type or "ema" in data_type:
                print("[DEBUG] Scaling all features")
                scaler = StandardScaler()
                scaler.fit(trainvalX)
                trainvalX = scaler.transform(trainvalX,copy=True)
            else:
                print("[DEBUG] Scaling static features")
                scaler = StandardScaler()
                scaler.fit(trainvalX[:,-6:])
                trainvalX[:,-6:] = scaler.transform(trainvalX[:,-6:],copy=True)

            if not DEBUG:
                MODDIR = train_mlp_model(RESDIR,trainvalX,trainvalY,
                                         data_type,label_type,hosp_data)

            (test1X,test1Y)       = load_data(DPATH,data_type,label_type,False,
                                              hosp_data,curr_feat,DEBUG=DEBUG)
            print("[Progress] test1X.shape    {}".format(test1X.shape))
            # Standardize data
            if "raw" in data_type or "ema" in data_type:
                print("[DEBUG] Scaling all features")
                test1X = scaler.transform(test1X,copy=True)
            else:
                print("[DEBUG] Scaling static features")
                test1X[:,-6:] = scaler.transform(test1X[:,-6:],copy=True)

            if not DEBUG:
                load_mlp_model_and_test(RESDIR,MODDIR,test1X,test1Y,
                                        data_type,label_type,hosp_data)